In [1]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
import openai
from llama_index.core.tools import FunctionTool
from llama_index.tools.google import GmailToolSpec
from llama_index.tools.google import GoogleCalendarToolSpec
from llama_index.tools.google import GoogleSearchToolSpec
# from llama_index.tools.wikipedia import WikipediaToolSpec
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

gmail_tools = GmailToolSpec().to_tool_list()
gcal_tools = GoogleCalendarToolSpec().to_tool_list()
gsearch_tools = GoogleSearchToolSpec(key=os.getenv('GOOGLE_SEARCH_API_KEY'), engine="engine").to_tool_list()
# wikipedia_tools = WikipediaToolSpec().to_tool_list(

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [3]:
for tool in [*gmail_tools, *gcal_tools, *gsearch_tools]:
    print(tool.metadata.name)
    print(tool.metadata.description)

load_data
load_data() -> List[llama_index.core.schema.Document]
Load emails from the user's account.
search_messages
search_messages(query: str, max_results: Optional[int] = None)
Searches email messages given a query string and the maximum number
        of results requested by the user
           Returns: List of relevant message objects up to the maximum number of results.

        Args:
            query[str]: The user's query
            max_results (Optional[int]): The maximum number of search results
            to return.
        
create_draft
create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str
Create and insert a draft email.
           Print the returned draft's message and id.
           Returns: Draft object, including draft id and message meta data.

        Args:
            to (Optional[str]): The email addresses to send the message to
            subject (Optional[str]): The subject for the event
            

In [4]:
# from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
from llama_index.core.tools.tool_spec.load_and_search import ( LoadAndSearchToolSpec,)

print("Wrapping " + gsearch_tools[0].metadata.name)
gsearch_load_and_search_tools = LoadAndSearchToolSpec.from_defaults( gsearch_tools[0],).to_tool_list()

print("Wrapping gmail " + gmail_tools[0].metadata.name)
gmail_load_and_search_tools = LoadAndSearchToolSpec.from_defaults( gmail_tools[0],).to_tool_list()

print("Wrapping google calendar " + gcal_tools[0].metadata.name)
gcal_load_and_search_tools = LoadAndSearchToolSpec.from_defaults( gcal_tools[0],).to_tool_list()

Wrapping google_search
Wrapping gmail load_data
Wrapping google calendar load_data


In [5]:
all_tools = [
    *gsearch_load_and_search_tools,
    *gmail_load_and_search_tools,
    *gcal_load_and_search_tools,
    *gcal_tools[1::],
    *gmail_tools[1::],
    *gsearch_tools[1::],
    multiply_tool,
    add_tool,
]

In [6]:
llm = OpenAI(model="gpt-3.5-turbo")
agent = OpenAIAgent.from_tools(tools=all_tools, llm=llm, verbose=True)

In [7]:
agent.chat(
  # "search google and find the email address for a dentist in toronto near bloor and dufferin"
  # "search google and find the phone number for a dentist in ridgewood near east ridgewood avenue"
    "search google and the sea is green"
)

Added user message to memory: search google and the sea is green
=== Calling Function ===
Calling function: google_search with args: {"query":"the sea is green"}
Got output: Content loaded! You can now search the information using read_google_search

=== Calling Function ===
Calling function: read_google_search with args: {"query":"the sea is green"}
Got output: API key not valid. Please pass a valid API key.



AgentChatResponse(response='I encountered an issue while trying to retrieve information from Google search. Could you please try again with a different query or provide more details so I can assist you further?', sources=[ToolOutput(content='Content loaded! You can now search the information using read_google_search', tool_name='google_search', raw_input={'args': (), 'kwargs': {'query': 'the sea is green'}}, raw_output='Content loaded! You can now search the information using read_google_search', is_error=False), ToolOutput(content='API key not valid. Please pass a valid API key.', tool_name='read_google_search', raw_input={'args': (), 'kwargs': {'query': 'the sea is green'}}, raw_output='API key not valid. Please pass a valid API key.', is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [8]:
agent.chat("What does google return for that search")

Added user message to memory: What does google return for that search
=== Calling Function ===
Calling function: google_search with args: {"query":"the sea is green"}
Got output: Content loaded! You can now search the information using read_google_search

=== Calling Function ===
Calling function: read_google_search with args: {"query":"the sea is green"}
Got output: There is no direct correlation between the provided context information and the query about the sea being green.



AgentChatResponse(response='Google search did not return any relevant information related to the query "the sea is green." If you have any other queries or need assistance with something else, feel free to let me know!', sources=[ToolOutput(content='Content loaded! You can now search the information using read_google_search', tool_name='google_search', raw_input={'args': (), 'kwargs': {'query': 'the sea is green'}}, raw_output='Content loaded! You can now search the information using read_google_search', is_error=False), ToolOutput(content='There is no direct correlation between the provided context information and the query about the sea being green.', tool_name='read_google_search', raw_input={'args': (), 'kwargs': {'query': 'the sea is green'}}, raw_output='There is no direct correlation between the provided context information and the query about the sea being green.', is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [9]:
agent.chat("write an email to send to john@johnrizzo.net asking if we can meet tomorrow at 2pm")

Added user message to memory: write an email to send to john@johnrizzo.net asking if we can meet tomorrow at 2pm
=== Calling Function ===
Calling function: create_draft with args: {"to":["john@johnrizzo.net"],"subject":"Meeting Request for Tomorrow at 2pm","message":"Hi John,\n\nI hope this email finds you well. I would like to schedule a meeting with you tomorrow at 2pm. Please let me know if this time works for you.\n\nLooking forward to hearing from you.\n\nBest regards,\n[Your Name]"}
Got output: Error: ('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'})

=== Calling Function ===
Calling function: create_draft with args: {"to":["john@johnrizzo.net"],"subject":"Meeting Request for Tomorrow at 2pm","message":"Hi John,\n\nI hope this email finds you well. I would like to schedule a meeting with you tomorrow at 2pm. Please let me know if this time works for you.\n\nLooking forward to hearing from you.\n\nB

AgentChatResponse(response='It appears there is an issue with creating the email draft due to a deleted OAuth client. Unfortunately, I am unable to proceed with creating the email at the moment. If you have any other requests or need assistance with something else, please let me know.', sources=[ToolOutput(content="Error: ('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'})", tool_name='create_draft', raw_input={'kwargs': {'to': ['john@johnrizzo.net'], 'subject': 'Meeting Request for Tomorrow at 2pm', 'message': 'Hi John,\n\nI hope this email finds you well. I would like to schedule a meeting with you tomorrow at 2pm. Please let me know if this time works for you.\n\nLooking forward to hearing from you.\n\nBest regards,\n[Your Name]'}}, raw_output=RefreshError('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'}), is_error=False),

In [10]:
# agent.chat("send an email to nonsleepr@gmail.com with the subject let's meet on Friday and the body this is the first email I've Ada has sent on my behalf.")

In [11]:
# agent.chat("What do I have on my calendar for the day")

In [12]:
agent.chat("How many emails have I received today")

Added user message to memory: How many emails have I received today
=== Calling Function ===
Calling function: get_date with args: {}
Got output: 2024-11-24

=== Calling Function ===
Calling function: load_data with args: {}
Got output: Error: ('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'})



AgentChatResponse(response='It seems there is an issue with accessing the email data due to a deleted OAuth client. Unfortunately, I am unable to retrieve the number of emails you have received today at the moment. If you have any other questions or need assistance with something else, please feel free to let me know.', sources=[ToolOutput(content='2024-11-24', tool_name='get_date', raw_input={'args': (), 'kwargs': {}}, raw_output=datetime.date(2024, 11, 24), is_error=False), ToolOutput(content="Error: ('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'})", tool_name='load_data', raw_input={'kwargs': {}}, raw_output=RefreshError('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'}), is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)